In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -q keras

In [ ]:
# !ls

In [0]:
import keras
import datetime
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, UpSampling2D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, Conv2DTranspose
import os
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
import numpy as np
from skimage import io, color
from keras.preprocessing import image
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback

image_size = 64
image_path = "data/"
def read_image(img_id, dir):
    try:
        img = load_img(dir + "/" + img_id, target_size=(image_size, image_size))
        img = img_to_array(img)
        return img
    except:
        return None

def show_image(image):
    plt.imshow(image/255.)
    plt.show()

def printOutput(file, output):
    for y in range(256):
        row = ""
        for x in range(256):
            row += str(output[y, x]) + " "
        file.write(row + "\n")
    file.close()
    
def meanCenter(trainData, dataToCenter):
  mean = np.mean(trainData, 0, dtype=np.float32)
  mean[:, :, 1:] = 0 #center only the lightness channel, which is the input
  return dataToCenter - mean

'''
X = []
files = os.listdir(image_path + "/Train/")[:2000]
for image in files:
    img = read_image(image, image_path + "/Train/")
    if not img is None:
        img = np.array(img, dtype=np.float32)
        X.append(img)

print(len(X))
X = np.array(X, dtype=np.float32)
Xtrain = (1.0/255)*X
Xtrain = color.rgb2lab(Xtrain)
print(Xtrain.shape)
Xtrain = meanCenter(Xtrain, Xtrain)
'''

test = []
files = os.listdir(image_path + "/Test/")[:200]
for image in files:
    img = read_image(image, image_path + "/Test/")
    if not img is None:
        img = np.array(img, dtype=np.float32)
        test.append(img)
test = np.array(test, dtype=np.float32)

test = test*(1.0/255)
test = color.rgb2lab(test)
#test = meanCenter(Xtrain, test)
test = test[:, :, :, 0]
test = test - 50
test = test/100
test = test.reshape(test.shape+(1,))

'''
#validation set
validation = []
files = os.listdir(image_path + "/Validation/")
for image in files:
    img = read_image(image, image_path + "/Validation/")
    if not img is None:
        img = np.array(img, dtype=np.float32)
        validation.append(img)
validation = np.array(validation, dtype=np.float32)

validation = validation*(1.0/255)
validation = color.rgb2lab(validation)
#test = meanCenter(Xtrain, test)
X_val = validation[:, :, :, 0]
X_val = X_val.reshape(X_val.shape+(1,))
Y_val = validation[:, :, :, 1:]/128
'''

In [0]:
now = datetime.datetime.now()

In [0]:
def convLayer(input, filters, kernel_size,  stride=1):
    return Conv2D(filters, kernel_size, padding="same", activation="relu", strides=stride)(input)

input_shape = (image_size, image_size, 1)

model_input = Input(shape = input_shape)

model_output = convLayer(model_input, 64, (3, 3))
model_output = convLayer(model_output, 64, (3, 3), stride=2)
model_output = BatchNormalization()(model_output)

model_output = convLayer(model_output, 128, (3, 3))
model_output = convLayer(model_output, 128, (3, 3), stride=2)
model_output = BatchNormalization()(model_output)

model_output = convLayer(model_output, 256, (3, 3))
model_output = convLayer(model_output, 256, (3, 3), stride=2)
model_output = BatchNormalization()(model_output)

model_output = convLayer(model_output, 512, (3, 3))
model_output = convLayer(model_output, 512, (3, 3))
model_output = BatchNormalization()(model_output)

model_output = convLayer(model_output, 512, (3, 3))
model_output = convLayer(model_output, 512, (3, 3))
model_output = BatchNormalization()(model_output)

model_output = UpSampling2D((2, 2))(model_output) #not sure if this or deconvolution
model_output = convLayer(model_output, 256, (3, 3))
model_output = BatchNormalization()(model_output)

model_output = UpSampling2D((2, 2))(model_output) 
model_output = convLayer(model_output, 64, (3, 3))
model_output = BatchNormalization()(model_output)

model_output = UpSampling2D((2, 2))(model_output) 
model_output = Conv2D(2, (3, 3), activation="tanh", padding="same")(model_output)

model = Model(inputs=model_input, outputs=model_output)

In [0]:
# model.load_weights("drive/app/output/my_model_weights2018-08-10 20:36.h5")
# if(not os.path.exists("/floyd/input/model/my_model_weights.h5")):
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #samplewise_center=False,
    shear_range=0.4,
    zoom_range=0.4,
    rotation_range=40,
    horizontal_flip=True,
    rescale=(1./255)
)

val_datagen = ImageDataGenerator(rescale=(1./255))

batch_size = 64
def batch_generator(batch_size):
    for batch in datagen.flow_from_directory(image_path+"/Train/",
                                             target_size=(image_size, image_size),
                                             class_mode="input",
                                             batch_size = batch_size):
        lab = color.rgb2lab(batch[0])
        X = lab[:, :, :, 0]
        X = X - 50
        X = X/100
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])
        
def val_batch_generator(batch_size):
    for batch in datagen.flow_from_directory(image_path+"/Validation/",
                                             target_size=(image_size, image_size),
                                             class_mode="input",
                                             batch_size = batch_size):
        lab = color.rgb2lab(batch[0])
        X = lab[:, :, :, 0]
        X = X - 50
        X = X/100
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])

model.summary()

outputDate = now.strftime("%Y-%m-%d %H:%M")
os.chdir("output")
os.mkdir(outputDate)
os.chdir(outputDate)

class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'currentWeights.h5'
            self.model.save_weights(name)
        self.batch += 1
        
checkpoint = ModelCheckpoint("best.hdf5",
                            monitor="accuracy",
                            verbose=1,
                            save_best_only=True,
                            mode="max")

every_20_batches = WeightsSaver(20)

every_10 = ModelCheckpoint("latest.hdf5",
                          monitor="accuracy",
                          verbose=1,
                          save_best_only=False,
                          mode='auto',
                          period=1)


tensorboard = TensorBoard(log_dir=".")
model.compile(loss='mean_squared_error',
            optimizer="adam",
            metrics=['accuracy'])

model.fit_generator(batch_generator(batch_size), callbacks=[tensorboard, checkpoint, every_10, every_20_batches], epochs=1, steps_per_epoch=5132, validation_data=val_batch_generator(batch_size)) #5132 steps per epoch


try:
    model.save_weights("model_weights.h5")
# else:
#     model.load_weights("/floyd/input/model/my_model_weights.h5")
except:
    print("Could not save")

model.save("model.h5")

In [0]:
# model.load_weights("drive/app/output/my_model_weights2018-08-10 20:36.h5")
# if(not os.path.exists("/floyd/input/model/my_model_weights.h5")):
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #samplewise_center=False,
    shear_range=0.4,
    zoom_range=0.4,
    rotation_range=40,
    horizontal_flip=True,
    rescale=(1./255)
)


batch_size = 64
def batch_generator(batch_size):
    for batch in datagen.flow_from_directory(image_path+"/Train/",
                                             target_size=(image_size, image_size),
                                             class_mode="input",
                                             batch_size = batch_size):
        lab = color.rgb2lab(batch[0])
        X = lab[:, :, :, 0]
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])

def val_batch_generator(batch_size):
    for batch in datagen.flow_from_directory(image_path+"/Validation/",
                                         target_size=(image_size, image_size),
                                         class_mode="input",
                                         batch_size = batch_size):
        lab = color.rgb2lab(batch[0])
        X = lab[:, :, :, 0]
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])
        
model.summary()

outputDate = now.strftime("%Y-%m-%d %H:%M")
os.chdir("output")
os.mkdir(outputDate)
os.chdir(outputDate)

class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'currentWeights.h5'
            self.model.save_weights(name)
        self.batch += 1
        
checkpoint = ModelCheckpoint("best.hdf5",
                            monitor="accuracy",
                            verbose=1,
                            save_best_only=True,
                            mode="max")

every_20_batches = WeightsSaver(20)

every_10 = ModelCheckpoint("latest.hdf5",
                          monitor="accuracy",
                          verbose=1,
                          save_best_only=False,
                          mode='auto',
                          period=1)

tensorboard = TensorBoard(log_dir=".")


model.load_weights("/content/drive/My Drive/app/output/2018-09-29 14:58/latest.hdf5")#manually change this, i know, i know

model.compile(loss='mean_squared_error',
            optimizer="adam",
            metrics=['accuracy'])

model.fit_generator(batch_generator(batch_size), callbacks=[tensorboard, checkpoint, every_10, every_20_batches], epochs=1, steps_per_epoch=5132, validation_data=val_batch_generator(batch_size)) #5132 steps per epoch


try:
    model.save_weights("model_weights.h5")
# else:
#     model.load_weights("/floyd/input/model/my_model_weights.h5")
except:
    print("Could not save")

model.save("model.h5")

In [0]:
model.load_weights("latest.hdf5")

model.compile(loss='mean_squared_error',
            optimizer="adam",
            metrics=['accuracy'])

In [0]:
# Test model
output = model.predict(test)
output = output * 128
print(output.shape)
# Output colorizations
for i in range(len(output)):
    cur = np.zeros((image_size, image_size, 3))
    cur[:,:,0] = test[i][:,:,0]
    cur[:,:,1:] = output[i]
    io.imsave(str(i) + ".png", color.lab2rgb(cur))